In [1]:
import pandas as pd
from geemap import ml
from sklearn import ensemble

import ee
import geemap

In [2]:
# Earth Engine 초기화
ee.Initialize()

In [3]:
url = "https://raw.githubusercontent.com/gee-community/geemap/master/examples/data/rf_example.csv"
df = pd.read_csv(url)

# 데이터프레임을 CSV 파일로 저장
output_file = "rf_example.csv"
df.to_csv(output_file, index=False)

df

,B2,B3,B4,B5,B6,B7,landcover
0,0.139846,0.114738,0.109982,0.119542,0.125795,0.105720,0
1,0.130316,0.109207,0.107499,0.140210,0.132006,0.108497,0
2,0.146690,0.135766,0.146550,0.225686,0.218105,0.167111,0
3,0.119413,0.108924,0.105196,0.144868,0.159775,0.122056,0
4,0.155492,0.139932,0.137486,0.151377,0.153771,0.133134,0
...,...,...,...,...,...,...,...
93,0.117331,0.092176,0.062548,0.020362,0.005813,0.004047,2
94,0.118353,0.093785,0.060253,0.020083,0.007317,0.004719,2
95,0.123362,0.095831,0.069663,0.027320,0.011386,0.008357,2
96,0.122907,0.100083,0.079527,0.024564,0.008570,0.006321,2


In [4]:
feature_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
label = "landcover"

In [5]:
X = df[feature_names]
y = df[label]
n_trees = 10

# scikit-learn을 사용하여 로컬에서 모델 훈련
rf = ensemble.RandomForestClassifier(n_trees).fit(X, y)

In [6]:
# scikit-learn 분류기 객체를 문자열 리스트로 변환
trees = ml.rf_to_strings(rf, feature_names)

In [7]:
print(len(trees))

10


In [8]:
print(trees[0])

1) root 62 9999 9999 (1.163265306122449)
  2) B7 <= 0.019053 20 0.0000 2 *
  3) B7 > 0.019053 62 0.6633 0
    6) B2 <= 0.117214 21 0.0000 1 *
    7) B2 > 0.117214 21 0.0000 0 *



In [9]:
# sk-learn 분류기를 GEE 분류기로 변환
ee_classifier = ml.strings_to_classifier(trees)
ee_classifier.getInfo()

{'type': 'Classifier.decisionTreeEnsemble',
 'treeStrings': ['1) root 62 9999 9999 (1.163265306122449)\n  2) B7 <= 0.019053 20 0.0000 2 *\n  3) B7 > 0.019053 62 0.6633 0\n    6) B2 <= 0.117214 21 0.0000 1 *\n    7) B2 > 0.117214 21 0.0000 0 *\n',
  '1) root 61 9999 9999 (2.1215051779528844)\n  2) B6 <= 0.047678 21 0.0000 2 *\n  3) B6 > 0.047678 61 0.6645 0\n    6) B5 <= 0.204988 40 0.4956 0\n      12) B2 <= 0.110891 5 0.0000 1 *\n      13) B2 > 0.110891 15 0.0000 0 *\n    7) B5 > 0.204988 40 0.4956 0\n      14) B6 <= 0.255315 20 0.4043 1\n        28) B4 <= 0.109414 13 0.0000 1 *\n        29) B4 > 0.109414 3 0.0000 0 *\n      15) B6 > 0.255315 4 0.0000 0 *\n',
  '1) root 65 9999 9999 (2.931862729101076)\n  2) B2 <= 0.124305 65 0.6562 2\n    4) B4 <= 0.057878 44 0.5199 2\n      8) B3 <= 0.070845 4 0.0000 1 *\n      9) B3 > 0.070845 26 0.3200 2\n        18) B7 <= 0.037645 19 0.0000 2 *\n        19) B7 > 0.037645 3 0.0000 1 *\n    5) B4 > 0.057878 44 0.5199 2\n      10) B7 <= 0.030050 4 0.

In [10]:
# GEE 분류기를 사용한 이미지 분류

# USGS Landsat 8 Collection 2 Tier 1 Raw Scenes
# https://developers.google.com/earth-engine/datasets/catalog/landsat-8
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1')

In [12]:
# cloud-free Landsat 8 TOA composite 생성
# https://developers.google.com/earth-engine/apidocs/ee-algorithms-landsat-simplecomposite
image = ee.Algorithms.Landsat.simpleComposite(
    collection=l8.filterDate('2022-01-01', '2022-12-31'), asFloat=True
)

In [11]:
Map = geemap.Map()

point = ee.Geometry.Point([126.2256, 36.6125])
Map.centerObject(point, 9)
Map.addLayer(
    image,
    {"bands": ['B7', 'B5', 'B3'], "min": 0.05, "max": 0.55, "gamma": 1.5},
    'image',
)
Map

Map(center=[36.6125, 126.22559999999999], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

In [13]:
classified = image.select(feature_names).classify(ee_classifier)

In [14]:
Map.addLayer(
    classified,
    {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},
    'classification',
)
Map

Map(bottom=51493.0, center=[36.6125, 126.22559999999999], controls=(WidgetControl(options=['position', 'transp…

In [15]:
# trees를 로컬에 저장
out_csv = "trees.csv"
ml.trees_to_csv(trees, out_csv)
another_classifier = ml.csv_to_classifier(out_csv)
classified = image.select(feature_names).classify(another_classifier)